In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='land-cover-clustering')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [ ]:
import geemap
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Function to mask clouds using the Sentinel-2 QA band
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0)
    )
    return image.updateMask(mask).divide(10000)

# Define the area of interest for the Galápagos Islands (Bounding box)
galapagos_aoi = ee.Geometry.BBox(-91.7, -1.1, -90.0, 0.2)

# Load and process the Sentinel-2 Surface Reflectance dataset
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(galapagos_aoi)
    .filterDate('2024-01-01', '2024-11-21')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

# Create a median composite to summarize the image collection
composite = dataset.median().clip(galapagos_aoi)

# Select the bands of interest for clustering
bands = ['B2', 'B3', 'B4', 'B8']  # Blue, Green, Red, NIR
clustering_image = composite.select(bands)

# Sample the image to create input for clustering
sample_points = clustering_image.sample(
    region=galapagos_aoi,
    scale=30,  # Scale in meters
    numPixels=5000,  # Number of points to sample
    geometries=False  # Don't include geometry to save memory
)

# Apply k-means clustering (unsupervised)
clusterer = ee.Clusterer.wekaKMeans(5).train(sample_points)

# Cluster the image
result = clustering_image.cluster(clusterer)

# Visualization parameters for the clustering result
vis_params = {
    'min': 0,
    'max': 4,  # Number of clusters - 1
    'palette': ['red', 'green', 'blue', 'yellow', 'purple']
}

# Display the result on the map
Map = geemap.Map()
Map.centerObject(galapagos_aoi, 9)
Map.addLayer(result.clip(galapagos_aoi), vis_params, "Clustering Result")
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, "True Color Composite")
Map


Map(center=[-0.44999719980416536, -90.85000000000007], controls=(WidgetControl(options=['position', 'transpare…